## Bitcoin Prediction Mini Project

In [3]:
# Importing libraries and modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from textblob import TextBlob

In [4]:
%pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [11]:
# Loading the bitcoin price data
price_df = pd.read_csv("../Datasets/bitcoin_price.csv")
price_df["Date"] = pd.to_datetime(price_df["Date"])
price_df.set_index("Date",inplace=True)
price_data = price_df[["Close"]]

# Loading the twitter sentiment data
tweet_df = pd.read_csv("../Datasets/btc_tweets.csv")
tweet_df["Date"] = pd.to_datetime(tweet_df["Date"])
tweet_df.set_index("Date", inplace=True)

In [13]:
print(price_df)
print(tweet_df)

                   Close
Date                    
2024-01-01  3.007426e+04
2024-01-02  5.979156e+04
2024-01-03  9.012004e+04
2024-01-04  1.200886e+05
2024-01-05  1.501125e+05
...                  ...
2024-12-26  1.082862e+07
2024-12-27  1.085887e+07
2024-12-28  1.088907e+07
2024-12-29  1.091905e+07
2024-12-30  1.094882e+07

[365 rows x 1 columns]
                                    Tweet
Date                                     
2024-01-01  Bitcoin is going to the moon!
2024-01-01   Bitcoin adoption increasing.
2024-01-01                HODLing strong!
2024-01-02   Bitcoin adoption increasing.
2024-01-02   Not sure about BTC's future.
...                                   ...
2024-12-29  Bitcoin is going to the moon!
2024-12-29             Bear market ahead?
2024-12-30          BTC might crash soon.
2024-12-30             Bear market ahead?
2024-12-30  Bitcoin is going to the moon!

[1095 rows x 1 columns]


In [15]:
# Generating daily sentiment score
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

tweet_df["Sentiment"] = tweet_df["Tweet"].apply(get_sentiment)
daily_sentiment = tweet_df[["Sentiment"]].resample("D").mean().fillna(0)

In [18]:
# Merge price and sentiment data
combined_df = pd.concat([price_data, daily_sentiment], axis=1).fillna(method="ffill")
combined_df

C:\Users\anish\AppData\Local\Temp\ipykernel_8900\631633025.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df = pd.concat([price_data, daily_sentiment], axis=1).fillna(method="ffill")


,Close,Sentiment
Date,,
2024-01-01,3.007426e+04,0.180556
2024-01-02,5.979156e+04,-0.041667
2024-01-03,9.012004e+04,-0.152778
2024-01-04,1.200886e+05,0.000000
2024-01-05,1.501125e+05,0.000000
...,...,...
2024-12-26,1.082862e+07,0.166667
2024-12-27,1.085887e+07,-0.111111
2024-12-28,1.088907e+07,0.305556


In [21]:
# Initialize scaler
scaler = MinMaxScaler()

# Fit and transform the combined dataframe
normalized_data = scaler.fit_transform(combined_df)

# Convert back to DataFrame with same index and column names
normalized_df = pd.DataFrame(normalized_data, index=combined_df.index, columns=combined_df.columns)

normalized_df

,Close,Sentiment
Date,,
2024-01-01,0.000000,0.596774
2024-01-02,0.002722,0.338710
2024-01-03,0.005499,0.209677
2024-01-04,0.008244,0.387097
2024-01-05,0.010994,0.387097
...,...,...
2024-12-26,0.988991,0.580645
2024-12-27,0.991761,0.258065
2024-12-28,0.994527,0.741935
